# About

Convert deeplabv3_mnv2_ade20k_train_2018_12_03 to Core ML

Use `MobilenetV2/MobilenetV2/input` instead of `ImageTensor` as input. The latency is the same as trial4. 

# Download model and setup enviroments

In [ ]:
!wget http://download.tensorflow.org/models/deeplabv3_mnv2_ade20k_train_2018_12_03.tar.gz
!tar -zxvf deeplabv3_mnv2_ade20k_train_2018_12_03.tar.gz

--2021-08-24 13:22:12--  http://download.tensorflow.org/models/deeplabv3_mnv2_ade20k_train_2018_12_03.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 64.233.189.128, 2404:6800:4008:c07::80
Connecting to download.tensorflow.org (download.tensorflow.org)|64.233.189.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26028742 (25M) [application/x-tar]
Saving to: ‘deeplabv3_mnv2_ade20k_train_2018_12_03.tar.gz’

deeplabv3_mnv2_ade2 100%[===================>]  24.82M  56.5MB/s    in 0.4s    

2021-08-24 13:22:13 (56.5 MB/s) - ‘deeplabv3_mnv2_ade20k_train_2018_12_03.tar.gz’ saved [26028742/26028742]

deeplabv3_mnv2_ade20k_train_2018_12_03/
deeplabv3_mnv2_ade20k_train_2018_12_03/model.ckpt.index
deeplabv3_mnv2_ade20k_train_2018_12_03/frozen_inference_graph.pb
deeplabv3_mnv2_ade20k_train_2018_12_03/model.ckpt.data-00000-of-00001


In [ ]:
!pip install coremltools==4.1
!pip install tfcoreml==2.0.0

     |████████████████████████████████| 3.4 MB 4.1 MB/s 
  Created wheel for attr: filename=attr-0.3.1-py3-none-any.whl size=2457 sha256=c26b665efa47f4a4911be5dff6c2e36b34becf9a02b6035a28183814dad5022b
  Stored in directory: /root/.cache/pip/wheels/3b/5d/58/41fbe92f47031641008bd8559ee89e58bf0f123f9c18dea1cb
Successfully built attr
     |████████████████████████████████| 44 kB 1.2 MB/s 
     |████████████████████████████████| 109.3 MB 12 kB/s 
     |████████████████████████████████| 488 kB 58.4 MB/s 
     |████████████████████████████████| 3.1 MB 21.2 MB/s 
     |████████████████████████████████| 50 kB 5.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
 

In [ ]:
import sys
print(sys.version)

import tensorflow as tf
print(tf.__version__)

import coremltools as ct
print(ct.__version__)

3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
1.14.0
4.1


In [ ]:
def load_frozen_graph(path):
  with tf.io.gfile.GFile(path, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

    return graph_def

def export_ops_name(the_graph, filename):
  with open(filename, "w") as text_file:
    ops = the_graph.get_operations()
    N = len(ops)
    for i in range(N):
      text_file.write('\n\nop id {} , op type: "{}"'.format(str(i), ops[i].type))
      
      text_file.write('\ninput(s):'),
      for x in ops[i].inputs:
        text_file.write("name = {}, shape: {}, ".format(x.name, x.get_shape()))
      
      text_file.write('\noutput(s):'),
      for x in ops[i].outputs:
        text_file.write("name = {}, shape: {},".format(x.name, x.get_shape()))
  print('Exported to:', filename)


input_file = '/content/deeplabv3_mnv2_ade20k_train_2018_12_03/frozen_inference_graph.pb'
model_name = 'deeplabv3_mnv2_ade20k_train_2018_12_03'
with tf.Session(graph=tf.Graph()) as sess:
  graph_def = load_frozen_graph(input_file)
  tf.import_graph_def(graph_def, name="")
  the_graph = tf.compat.v1.get_default_graph()
  export_ops_name(the_graph, f'{model_name}_ops.txt')



Exported to: deeplabv3_mnv2_ade20k_train_2018_12_03_ops.txt


In [ ]:
input_name = "MobilenetV2/MobilenetV2/input"
output_name = "ArgMax"

input_w = 513
input_h = 513

tf_frozen_graph_file = 'deeplabv3_mnv2_ade20k_train_2018_12_03/frozen_inference_graph.pb'
optimized_frozen_graph_file = 'optimized_frozen_inference_graph.pb'

In [ ]:
from tensorflow.python.tools import strip_unused_lib
from tensorflow.python.framework import dtypes
from tensorflow.python.platform import gfile


def print_node_name(graph_def):
  nodes = [n.name for n in graph_def.node]
  print('nodes', len(nodes), nodes)
  print(nodes[0], '->', nodes[-1])


def optimize_graph(graph_def, 
                   input_node_names, 
                   output_node_names,
                   frozen_model_file):

  print('nodes before strip_unused_lib')
  print_node_name(graph_def)
  
  graph_def = strip_unused_lib.strip_unused(
          input_graph_def = graph_def,
          input_node_names = input_node_names,
          output_node_names = output_node_names,
          placeholder_type_enum = dtypes.float32.as_datatype_enum)
  
  print('nodes after strip_unused_lib')
  print_node_name(graph_def)

  with gfile.GFile(frozen_model_file, "wb") as f:
    f.write(graph_def.SerializeToString())

 
graph_def = load_frozen_graph(input_file)
optimize_graph(graph_def,
              input_node_names=[input_name], 
              output_node_names=[output_name],
              frozen_model_file=optimized_frozen_graph_file)
 

nodes before strip_unused_lib
nodes 1034 ['ImageTensor', 'Shape', 'strided_slice/stack', 'strided_slice/stack_1', 'strided_slice/stack_2', 'strided_slice', 'Squeeze', 'Cast', 'Shape_1', 'strided_slice_1/stack', 'strided_slice_1/stack_1', 'strided_slice_1/stack_2', 'strided_slice_1', 'strided_slice_2/stack', 'strided_slice_2/stack_1', 'strided_slice_2/stack_2', 'strided_slice_2', 'sub/x', 'sub', 'Maximum/y', 'Maximum', 'add', 'sub_1/x', 'sub_1', 'Maximum_1/y', 'Maximum_1', 'add_1', 'Reshape/tensor', 'Reshape/shape', 'Reshape', 'Rank', 'Equal/y', 'Equal', 'Assert/Assert/data_0', 'Assert/Assert/data_1', 'Assert/Assert', 'sub_2', 'Shape_2', 'strided_slice_3/stack', 'strided_slice_3/stack_1', 'strided_slice_3/stack_2', 'strided_slice_3', 'strided_slice_4/stack', 'strided_slice_4/stack_1', 'strided_slice_4/stack_2', 'strided_slice_4', 'GreaterEqual', 'Assert_1/Assert/data_0', 'Assert_1/Assert', 'GreaterEqual_1', 'Assert_2/Assert/data_0', 'Assert_2/Assert', 'sub_3/y', 'sub_3', 'sub_4', 'sub_5

In [ ]:
import tfcoreml
import coremltools as ct

# Convert the TensorFlow model to Core ML.

input_path = optimized_frozen_graph_file
output_path = "DeepLabV3_no_metadata.mlmodel"

input_tensor = "MobilenetV2/MobilenetV2/input:0"
input_name = "MobilenetV2/MobilenetV2/input__0"
output_tensor = "ArgMax:0"

coreml_model = tfcoreml.convert(
    tf_model_path=input_path,
    mlmodel_path=output_path,
    output_feature_names=[output_tensor],
    input_name_shape_dict={input_tensor: [1, 513, 513, 3]},
    image_input_names=input_name)



Loading the TF graph...
Graph Loaded.
Now finding ops in the TF graph that can be dropped for inference
Done.
Now starting translation to CoreML graph.
Automatic shape interpretation succeeded for input blob MobilenetV2/MobilenetV2/input:0
1/933: Analysing op name: ArgMax/dimension ( type:  Const )
2/933: Analysing op name: strided_slice_6/stack_2 ( type:  Const )
3/933: Analysing op name: strided_slice_6/stack_1 ( type:  Const )
4/933: Analysing op name: strided_slice_6/stack ( type:  Const )
5/933: Analysing op name: Shape_4 ( type:  Const )
6/933: Analysing op name: strided_slice_6 ( type:  StridedSlice )
7/933: Analysing op name: ResizeBilinear_1/size ( type:  Const )
8/933: Analysing op name: logits/semantic/biases ( type:  Const )
9/933: Analysing op name: logits/semantic/biases/read ( type:  Identity )
10/933: Analysing op name: logits/semantic/weights ( type:  Const )
11/933: Analysing op name: logits/semantic/weights/read ( type:  Identity )
12/933: Analysing op name: decoder

In [ ]:
from coremltools.models.neural_network import quantization_utils

# allowed values of nbits = 16, 8, 7, 6, ...., 1
spec = quantization_utils.quantize_weights(coreml_model, nbits=8)

Quantizing using linear quantization
Optimizing Neural Network before Quantization:
Fused MobilenetV2/Conv/Conv2D:0->MobilenetV2/Conv/BatchNorm/FusedBatchNorm:0
Fused MobilenetV2/expanded_conv/depthwise/depthwise:0->MobilenetV2/expanded_conv/depthwise/BatchNorm/FusedBatchNorm:0
Fused MobilenetV2/expanded_conv/project/Conv2D:0->MobilenetV2/expanded_conv/project/BatchNorm/FusedBatchNorm:0
Fused MobilenetV2/expanded_conv_1/expand/Conv2D:0->MobilenetV2/expanded_conv_1/expand/BatchNorm/FusedBatchNorm:0
Fused MobilenetV2/expanded_conv_1/depthwise/depthwise:0->MobilenetV2/expanded_conv_1/depthwise/BatchNorm/FusedBatchNorm:0
Fused MobilenetV2/expanded_conv_1/project/Conv2D:0->MobilenetV2/expanded_conv_1/project/BatchNorm/FusedBatchNorm:0
Fused MobilenetV2/expanded_conv_2/expand/Conv2D:0->MobilenetV2/expanded_conv_2/expand/BatchNorm/FusedBatchNorm:0
Fused MobilenetV2/expanded_conv_2/depthwise/depthwise:0->MobilenetV2/expanded_conv_2/depthwise/BatchNorm/FusedBatchNorm:0
Fused decoder/feature_pro

In [ ]:
# Fill in the descriptions and metadata.
spec.description.metadata.versionString = "2021-08-11"
spec.description.metadata.shortDescription = "deeplabv3_mnv2_ade20k_train_2018_12_03"
spec.description.metadata.author = "Converted by Anh"


# Rename inputs and outputs.
old_input_name = "MobilenetV2__MobilenetV2__input__0"
new_input_name = "image"

old_output_name = "ArgMax__0"
new_output_name = "scores"

ct.utils.rename_feature(spec, old_input_name, new_input_name)
ct.utils.rename_feature(spec, old_output_name, new_output_name)

spec.description.input[0].shortDescription = "Input image"
spec.description.output[0].shortDescription = "Segmentation map"

# Fix up the output shape and make it INT32.
spec.description.output[0].type.multiArrayType.shape.append(513)
spec.description.output[0].type.multiArrayType.shape.append(513)
spec.description.output[0].type.multiArrayType.dataType = ct.proto.FeatureTypes_pb2.ArrayFeatureType.INT32


In [ ]:
print(spec.description)

input {
  name: "image"
  shortDescription: "Input image"
  type {
    imageType {
      width: 513
      height: 513
      colorSpace: RGB
    }
  }
}
output {
  name: "scores"
  shortDescription: "Segmentation map"
  type {
    multiArrayType {
      shape: 513
      shape: 513
      dataType: INT32
    }
  }
}
metadata {
  shortDescription: "deeplabv3_mnv2_ade20k_train_2018_12_03"
  versionString: "2021-08-11"
  author: "Converted by Anh"
}



In [ ]:
labels = """
wall
building, edifice
sky
floor, flooring
tree
ceiling
road, route
bed 
windowpane, window 
grass
cabinet
sidewalk, pavement
person, individual, someone, somebody, mortal, soul
earth, ground
door, double door
table
mountain, mount
plant, flora, plant life
curtain, drape, drapery, mantle, pall
chair
car, auto, automobile, machine, motorcar
water
painting, picture
sofa, couch, lounge
shelf
house
sea
mirror
rug, carpet, carpeting
field
armchair
seat
fence, fencing
desk
rock, stone
wardrobe, closet, press
lamp
bathtub, bathing tub, bath, tub
railing, rail
cushion
base, pedestal, stand
box
column, pillar
signboard, sign
chest of drawers, chest, bureau, dresser
counter
sand
sink
skyscraper
fireplace, hearth, open fireplace
refrigerator, icebox
grandstand, covered stand
path
stairs, steps
runway
case, display case, showcase, vitrine
pool table, billiard table, snooker table
pillow
screen door, screen
stairway, staircase
river
bridge, span
bookcase
blind, screen
coffee table, cocktail table
toilet, can, commode, crapper, pot, potty, stool, throne
flower
book
hill
bench
countertop
stove, kitchen stove, range, kitchen range, cooking stove
palm, palm tree
kitchen island
computer, computing machine, computing device, data processor, electronic computer, information processing system
swivel chair
boat
bar
arcade machine
hovel, hut, hutch, shack, shanty
bus, autobus, coach, charabanc, double-decker, jitney, motorbus, motorcoach, omnibus, passenger vehicle
towel
light, light source
truck, motortruck
tower
chandelier, pendant, pendent
awning, sunshade, sunblind
streetlight, street lamp
booth, cubicle, stall, kiosk
television, television receiver, television set, tv, tv set, idiot box, boob tube, telly, goggle box
airplane, aeroplane, plane
dirt track
apparel, wearing apparel, dress, clothes
pole
land, ground, soil
bannister, banister, balustrade, balusters, handrail
escalator, moving staircase, moving stairway
ottoman, pouf, pouffe, puff, hassock
bottle
buffet, counter, sideboard
poster, posting, placard, notice, bill, card
stage
van
ship
fountain
conveyer belt, conveyor belt, conveyer, conveyor, transporter
canopy
washer, automatic washer, washing machine
plaything, toy
swimming pool, swimming bath, natatorium
stool
barrel, cask
basket, handbasket
waterfall, falls
tent, collapsible shelter
bag
minibike, motorbike
cradle
oven
ball
food, solid food
step, stair
tank, storage tank
trade name, brand name, brand, marque
microwave, microwave oven
pot, flowerpot
animal, animate being, beast, brute, creature, fauna
bicycle, bike, wheel, cycle 
lake
dishwasher, dish washer, dishwashing machine
screen, silver screen, projection screen
blanket, cover
sculpture
hood, exhaust hood
sconce
vase
traffic light, traffic signal, stoplight
tray
ashcan, trash can, garbage can, wastebin, ash bin, ash-bin, ashbin, dustbin, trash barrel, trash bin
fan
pier, wharf, wharfage, dock
crt screen
plate
monitor, monitoring device
bulletin board, notice board
shower
radiator
glass, drinking glass
clock
flag
"""
 
labels_list = labels.split('\n')
print(len(labels_list))
print(labels_list)
assert(len(labels_list) == 152)

152
['', 'wall', 'building, edifice', 'sky', 'floor, flooring', 'tree', 'ceiling', 'road, route', 'bed ', 'windowpane, window ', 'grass', 'cabinet', 'sidewalk, pavement', 'person, individual, someone, somebody, mortal, soul', 'earth, ground', 'door, double door', 'table', 'mountain, mount', 'plant, flora, plant life', 'curtain, drape, drapery, mantle, pall', 'chair', 'car, auto, automobile, machine, motorcar', 'water', 'painting, picture', 'sofa, couch, lounge', 'shelf', 'house', 'sea', 'mirror', 'rug, carpet, carpeting', 'field', 'armchair', 'seat', 'fence, fencing', 'desk', 'rock, stone', 'wardrobe, closet, press', 'lamp', 'bathtub, bathing tub, bath, tub', 'railing, rail', 'cushion', 'base, pedestal, stand', 'box', 'column, pillar', 'signboard, sign', 'chest of drawers, chest, bureau, dresser', 'counter', 'sand', 'sink', 'skyscraper', 'fireplace, hearth, open fireplace', 'refrigerator, icebox', 'grandstand, covered stand', 'path', 'stairs, steps', 'runway', 'case, display case, show

In [ ]:
import json
from numpy import loadtxt

labels_json = {"labels": labels_list}

# load the model
mlmodel = ct.models.MLModel(spec)
mlmodel.user_defined_metadata["com.apple.coreml.model.preview.type"] = "imageSegmenter"
mlmodel.user_defined_metadata['com.apple.coreml.model.preview.params'] = json.dumps(labels_json)

mlmodel.save("DeepLabV3_ADE20K.mlmodel")
print(mlmodel)

input {
  name: "image"
  shortDescription: "Input image"
  type {
    imageType {
      width: 513
      height: 513
      colorSpace: RGB
    }
  }
}
output {
  name: "scores"
  shortDescription: "Segmentation map"
  type {
    multiArrayType {
      shape: 513
      shape: 513
      dataType: INT32
    }
  }
}
metadata {
  shortDescription: "deeplabv3_mnv2_ade20k_train_2018_12_03"
  versionString: "2021-08-11"
  author: "Converted by Anh"
  userDefined {
    key: "com.apple.coreml.model.preview.params"
    value: "{\"labels\": [\"\", \"wall\", \"building, edifice\", \"sky\", \"floor, flooring\", \"tree\", \"ceiling\", \"road, route\", \"bed \", \"windowpane, window \", \"grass\", \"cabinet\", \"sidewalk, pavement\", \"person, individual, someone, somebody, mortal, soul\", \"earth, ground\", \"door, double door\", \"table\", \"mountain, mount\", \"plant, flora, plant life\", \"curtain, drape, drapery, mantle, pall\", \"chair\", \"car, auto, automobile, machine, motorcar\", \"water\",